# Kalman Filter Simple Steps

First, let's start with 2-D.

1. Calculate Kalman Gain. $$ KG = \frac{E_{EST}}{E_{EST} + E_{MEA}}$$ where $E_{EST}$ means an Error in Estimate and $E_{MEA}$ means an error in measurement. Please notice $$ 0 <= KG <= 1 $$
2. Compute a next Estimate. $$EST_{t} = EST_{t - 1} + KG(MEA - EST_{t -1})$$
3. Compute a new error in estimate. 
\begin{align*}
E_{EST_t} &= \frac{E_{MEA} \cdot E_{EST_{t-1}} } { E_{MEA} + E_{EST_{t-1}}} \\
                &= (1 - KG) \cdot E_{EST_{t-1}}
\end{align*}

## Simple Example
- Suppose there is a thermostat that measure a room's temperature
- True Temperature: 72
- Initial Guess(Estimate): 60
- Measurement: 75
- Error in Estimate: 2
- Error in Measurement: 4

In [1]:
import numpy as np

In [2]:
def get_kalman_gain(err_estimate, err_measure):
    return err_estimate / (err_estimate + err_measure)

def get_new_estimate(KG, previous_estimate, measurement):
    return previous_estimate + KG * (measurement - previous_estimate)

def get_new_estimate_error(KG, previous_error):
    return (1 - KG) * previous_error

#### We can see every several steps it approaches to the true temperature eventually.

In [3]:
estimate = 60
size = 1000
measurement = np.random.normal(loc = 72, scale = 4, size = size)
error_estimate = 2
error_measure = 4

for i, measurement in enumerate(measurement):
    KG = get_kalman_gain(error_estimate, error_measure)
    estimate = get_new_estimate(KG, estimate, measurement)
    error_estimate = get_new_estimate_error(KG, error_estimate)
        
    msg = f"""

Kalman Gain: {KG}
Measurement: {measurement}
Estimate: {estimate}
----------------------
True Value: {72}
----------------------
Error in Estimate: {error_estimate}
    """
    
    
    if i % size // 10 == 0 or i == size -1 or i == 0:
        #print(msg)
        print(f"Estimate: {estimate}")
    

Estimate: 62.59544454771902
Estimate: 62.07249802797206
Estimate: 62.16521641329086
Estimate: 63.62479048276855
Estimate: 64.88640511542268
Estimate: 65.34378468135901
Estimate: 65.58708715836822
Estimate: 66.21872291380232
Estimate: 67.37896802651733
Estimate: 67.412325025897
Estimate: 72.07481349758007


The easy way to see how it works is to visualize in a table

Assume the initial estimate is 68 and measurements are [75, 71, 70].

|     | $MEA$ | $E_{MEA}$ | $EST$ | $E_{EST_{t-1}}$ | $KG$ | $E_{EST_t}$ |
|----|-----------|----------------|---------|---------------------|----------|-----------------|
|$t-1$|              |                     |     68       |       2                    |            |                      |      
|$t    $|      75        |         4            |       70.33     |                           |     0.33          |   1.33                     |
|$t+1$|      71        |        4             |     70.50       |                           |     0.25         |    1.00                  |
|$t+2$|       70       |       4              |       70.40     |                           |     0.20          |    0.80                  |

## Multi-dimensional Kalman Filter

First of all, I'm going to write down the whole process first and explain each step in details later.

1. There is an initial state X such as $$\mathbf{X_0} = \begin{bmatrix} 
                                                                    X_0 \\
                                                                    P_0
                                                           \end{bmatrix}         $$

2. Then, we can get a new state or predicted state:
$$\begin{align}
\mathbf{X_{t_p}} &= \mathbf{A} \cdot \mathbf{X_{t-1}} + \mathbf{B} \cdot \mathbf{u_t} + \mathbf{w_t} \\
\mathbf{P_{t_p}} &= \mathbf{A} \cdot  \mathbf{ P_{t-1}} \mathbf{A^T} + \mathbf{Q_t} \\
\end{align}$$
where 
    - $\mathbf{X_{t_p}}$: New Predicted State
    - $\mathbf{u_k}$: Control Variable like acceleration, gravity, etc.
    - $\mathbf{w}$: Predicted State Noise Matrix
    - $\mathbf{Q}$: Process noise Covariance Matrix    
    - $\mathbf{P}$: Process Covariance Matrix (Represents an error in estimate)    
    - $\mathbf{A}, \mathbf{B}, ...$: a helper matrix (State Transition Matrix, Control Input Matrix, ...)

3. Update Kalman Gain:
$$
    \begin{align}
        \mathbf{K} &= \frac{\mathbf{P_{t_p}} \cdot \mathbf{H}} {\mathbf{H} \cdot \mathbf{P_{t_p}} \cdot \mathbf{H^T} + \mathbf{R}} \\
        \mathbf{X_t} &= \mathbf{X_{t_p}} + \mathbf{K}[\mathbf{Y} - \mathbf{H} \cdot \mathbf{X_{t_p}} ]
    \end{align}
$$
where
$$
    \begin{align}
        \mathbf{Y_t} &= \mathbf{C} \cdot \mathbf{X_{t_m}} + \mathbf{Z_t}
     \end{align}
$$
where
    - $\mathbf{R}$: Sensor Noise Covariance Matrix (Error in Measurement)
    - $\mathbf{Z_t}$: Uncertainty
    - $\mathbf{C}$: the transformation matrix that maps the state vector parameters into the measurement domain
4. Update new $\mathbf{P_t}$ and $\mathbf{X_t}$:
$$
    \begin{align}
        \mathbf{P_t} &= \left( \mathbf{I} - \mathbf{K} \cdot \mathbf{H} \right) \cdot \mathbf{P_{t_p}} \\
        \mathbf{X_t}
    \end{align}
$$
5. Repeat from step 2 again.

## Covariance Matrix Example

Covariance matrix plays a key role in kalman filter. So, let's quickly review one example.

Suppose there are two random variables, $\mathbf{X}$ and $\mathbf{Y}$.

\begin{align*}
    Var(X) &= \frac{1}{N} \sum_i^N (x_i - \bar{x_i})^2 \\ \\
    Cov(X, Y) &= \frac{1}{N} \sum_i^N (x_i - \bar{x_i})(y_i - \bar{y_i})     
\end{align*}

$Cov(X, Y)$ can be also written as $\sigma_{XY}$.

#### Example. 

Each column means one randome variable, say $\mathbf{X}, \mathbf{Y}, \mathbf{Z}$.

In [30]:
A = np.array([[ 4.  ,  2.  ,  0.6    ],
                       [ 4.2 ,  2.1 ,  0.59],
                       [ 3.9 ,  2.  ,  0.56 ],
                       [ 4.3 ,  2.1 ,  0.62],
                       [ 4.1 ,  2.3 ,  0.63]])
mean_list = np.mean(A, axis=0)
mean_list

array([ 4.1,  2.1,  0.6])

In [31]:
A_mean_subtracted = A - mean_list
A_mean_subtracted

array([[-0.1 , -0.1 ,  0.  ],
       [ 0.1 ,  0.  , -0.01],
       [-0.2 , -0.1 , -0.04],
       [ 0.2 ,  0.  ,  0.02],
       [ 0.  ,  0.2 ,  0.03]])

In [59]:
var_X = np.var(A, axis=0)
cov_XY = np.mean(A_mean_subtracted[:, 0] * A_mean_subtracted[:, 1])
cov_XZ = np.mean(A_mean_subtracted[:, 0] * A_mean_subtracted[:, 2])
cov_YZ = np.mean(A_mean_subtracted[:, 1] * A_mean_subtracted[:, 2])

print ("Variance: {}".format(var_X))
print ("Cov(X, Y): {:.4f}".format(cov_XY))
print ("Cov(X, Z): {:.4f}".format(cov_XZ))
print ("Cov(Y, Z): {:.4f}".format(cov_YZ))

Variance: [ 0.02    0.012   0.0006]
Cov(X, Y): 0.0060
Cov(X, Z): 0.0022
Cov(Y, Z): 0.0020


In [60]:
np.cov(A, rowvar=False, bias=True) # check if we get the same value 

array([[ 0.02  ,  0.006 ,  0.0022],
       [ 0.006 ,  0.012 ,  0.002 ],
       [ 0.0022,  0.002 ,  0.0006]])

#### Another way to calculate covariance matrix by linear algebra

Say there is a matrix with M obeservations and N variables. Then, 

$$ \text{Covariance Matrix} =  \frac{\mathbb{a}^T \mathbb{a}} {M}  $$

where $$a = A - ones(M, M) \cdot A \cdot \left( \frac{1}{M} \right)$$

a is called a deviation matrix

In [78]:
a = A - np.ones(shape = (A.shape[0], A.shape[0])).dot(A) / A.shape[0]
a.transpose().dot(a)/A.shape[0]

array([[ 0.02  ,  0.006 ,  0.0022],
       [ 0.006 ,  0.012 ,  0.002 ],
       [ 0.0022,  0.002 ,  0.0006]])

In [83]:
## or use broadcast
a = A - np.mean(A, axis=0)
a.transpose().dot(a)/A.shape[0]

array([[ 0.02  ,  0.006 ,  0.0022],
       [ 0.006 ,  0.012 ,  0.002 ],
       [ 0.0022,  0.002 ,  0.0006]])

## Real Example

1. $x_0 = 50$ $m$
2. $\dot{x_0} = v_0 = 5$ $m/sec$
3. $a_0 = 2$ $m/sec^2$

There is a $X$ such that
$$ X = \begin{bmatrix} x\\ \dot{x} \end{bmatrix} $$

And process variation standard deviation is given

1. $\sigma_x = 0.5m$
2. $\sigma_{\dot{x}} = 0.2m/sec$
3. For simplicity, we just make a guess such that $\sigma_{x\dot{x}} = \sigma_x \cdot \sigma_{\dot{x}} = 0.1$
    - However, it's important to know $$Correlation(X, Y) = \frac{Cov(X, Y)}{SD(X) SD(Y)}$$ we're just making a guess here.

There is a $P$($P$ stands for Process or Estimate Error) such that
$$ 
\begin{align}
P &= \begin{bmatrix}
                    \sigma_x^2    &    \sigma_{x\dot{x}} \\
                    \sigma_{\dot{x}x}    &    \sigma_{\dot{x}}^2 \\
            \end{bmatrix} \\
 P_0   &= \begin{bmatrix}
                    0.25 & 0.1 \\
                    0.1 & 0.04
            \end{bmatrix} \\
\end{align}
$$

## Tracking an airplane example

Given Information:

- $x_0 = 4000\ m$
- $y_0 = 3000\ m$
- $v_{x_0} = 280\ m/\sec$
- $v_{y_0} = 120\ m/\sec$

Observations:

|       | $x_0$                |  $v_{x_0}$          |
|-----|-------- |-----------------|
|0   |4,000 | 280|
|1   |4,260 | 282|
|2   |4,550 | 285|
|3   |4,860 | 286|
|4   |5,110 | 290|


Initial Conditions:

| $a_x$ | $v_x$ | $\Delta t$ | $\Delta x $ |
|---------|---------|----------------|------------------|
|2 $m/sec^2$|280 $m/sec$|1 $sec$|25 $m$|

Process Errors in Process Covariance Matrix:
\begin{align}
        \Delta P_x       &= 20 \\
        \Delta P_{v_x} &= 5
\end{align}

Observation Errors:

\begin{align}
    \Delta x       &= 25 &\text{ (observation error) }\\
    \Delta v_{x} &= 6
\end{align}